In [111]:
import pyspark
from pyspark.sql.functions import least, udf, pandas_udf, PandasUDFType, col,\
split, trim, regexp_replace, when, translate, substring, lower
from pyspark.sql.types import StringType, FloatType, IntegerType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [98]:
## this dataset is around 11Gb and has 35 million rows approx

data = spark.read.option("header", "true").csv("s3://big-datasets-over-gb/seattle_library/library-collection-inventory.csv")
data.tail(4)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(BibNum='3045405', Title='Ignite : a Defy novel / Sara B. Larson.', Author='Larson, Sara B.', ISBN='0545644747, 9780545644747', PublicationYear='2015.', Publisher='Scholastic Press,', Subjects='Magic Fiction, Kings and rulers Fiction, Conspiracies Fiction, Adventure stories, Young adult fiction', ItemType='acbk', ItemCollection='nyfic', FloatingItem='NA', ItemLocation='nga', ReportDate='2019-11-01T00:00:00.000', ItemCount='1'), Row(BibNum='3386344', Title='Left : a love story / Mary Hogan.', Author='Hogan, Mary, 1957-', ISBN='006267837X, 9780062678379', PublicationYear='[2018]', Publisher='William Morrow, an imprint of HarperCollins Publishers,', Subjects='Married people Fiction, Dementia Fiction, Medical fiction', ItemType='acbk', ItemCollection='canew', FloatingItem='NA', ItemLocation='cen', ReportDate='2019-11-01T00:00:00.000', ItemCount='1'), Row(BibNum='3438763', Title="Beyond curry : Indian cookbook : a culinary journey through India / Denise D'silva Sankhé.", Author="Sankhé,

In [104]:
select_cols = ['Title', 'Author','PublicationYear', 'Publisher', 'Subjects', 
               'ItemType', 'ItemCollection', 'FloatingItem', 'ItemLocation']

df = data.select(*select_cols)\
.withColumnRenamed('Publicationyear', 'Year')\
.withColumnRenamed('ItemLocation', 'ItemLoc')\
.drop_duplicates()\

df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Title: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- PublishedYr: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Subjects: string (nullable = true)
 |-- ItemType: string (nullable = true)
 |-- ItemCollection: string (nullable = true)
 |-- FloatingItem: string (nullable = true)
 |-- ItemLoc: string (nullable = true)

In [105]:
df.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Title='The darkest star / Jennifer L. Armentrout.', Author='Armentrout, Jennifer L.', PublishedYr='2018.', Publisher='Tor Teen,', Subjects='Extraterrestrial beings Juvenile fiction, Secrecy Juvenile fiction, Love Fiction, Science fiction', ItemType='acbk', ItemCollection='nyfic', FloatingItem='NA', ItemLoc='mag'), Row(Title='The scourge / Jennifer A. Nielsen.', Author='Nielsen, Jennifer A.', PublishedYr='2016.', Publisher='Scholastic Press,', Subjects='Epidemics Juvenile fiction, Communicable diseases Juvenile fiction, Conspiracies Juvenile fiction, Government Resistance to Juvenile fiction, Best friends Juvenile fiction, Adventure stories, Epidemics Fiction, Diseases Fiction, Conspiracies Fiction, Government Resistance to Fiction, Best friends Fiction, Friendship Fiction, Adventure and adventurers Fiction, Adventure fiction', ItemType='acbk', ItemCollection='nyfic', FloatingItem='NA', ItemLoc='col'), Row(Title='Remember Little Bighorn : Indians, soldiers, and scouts tell their st

In [120]:
## lets reparition the data again as we have filtered 

repartitioned_df = df.withColumn('Year', substring(trim(regexp_replace("PublishedYr",r'[^0-9]',' ')),1,4).cast(IntegerType()))\
.filter(col('Year').between(2015, 2020))\
.coalesce(1)\
.repartition(6)

# lets cache the data so 

repartitioned_df.persist()

repartitioned_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----------+--------------------+--------------------+--------+--------------+------------+-------+----+
|               Title|              Author|PublishedYr|           Publisher|            Subjects|ItemType|ItemCollection|FloatingItem|ItemLoc|Year|
+--------------------+--------------------+-----------+--------------------+--------------------+--------+--------------+------------+-------+----+
|Waiting for Eden ...|    Ackerman, Elliot|      2018.|    Alfred A. Knopf,|Veterans Fiction,...|    acbk|         nanew|          NA|    cap|2018|
|Alexander Hamilto...|       Kelley, K. C.|      2017.|Children's Press,...|Hamilton Alexande...|    jcbk|         ncenf|          NA|    swt|2017|
|Sloppy joe stink-...|       Scroggs, Kirk|      2015.|Little, Brown and...|School lunchrooms...|    jcbk|         ccfic|          NA|    cen|2015|
|Pirates of the Ca...|                null|      2017.|Distributed by Bu...|Sparrow Jack Fict...|   acdvd|      

In [122]:
print(f"Number of rows: {repartitioned_df.count()} with {repartitioned_df.rdd.getNumPartitions()} partitions")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of rows: 525027 with 6 partitions

In [116]:
repartitioned_df.filter(lower(col('Subjects')).contains('sport'))\
.select('Title','Author','Subjects', 'Publisher')\
.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+
|               Title|              Author|            Subjects|           Publisher|
+--------------------+--------------------+--------------------+--------------------+
|Sports are fantas...|Könnecke, Ole, 1961-|Sports stories, R...|        Gecko Press,|
|Careers if you li...|Kallen, Stuart A....|Sports Vocational...|ReferencePoint Pr...|
|Thanks a lot, Joh...|  Parkhurst, Johanna|Hockey players Ve...|  Harmony Ink Press,|
|Home run, touchdo...|         Landry, Leo|Sports Juvenile p...|Henry Holt and Co...|
|Heidi Heckelbeck ...|        Coven, Wanda|Sports Fiction, S...|       Little Simon,|
|Walter Camp : foo...|  Des Jardins, Julie|Camp Walter 1859 ...|Oxford University...|
|The way downtown ...|    Gertsberg, Inna,|Local transit Juv...|     Kids Can Press,|
|Aaron is a good s...|Eastman, P. D. (P...|Ability Fiction, ...|       Random House,|
|Ali : a life / Jo...|       Eig, Jonathan|Ali Muhamma

In [107]:
# which years were most books published 


df_year_count = repartitioned_df.groupBy(col('Year')).count().orderBy(col('count').desc_nulls_last())
df_year_count.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+
|Year| count|
+----+------+
|2017|133647|
|2016|114953|
|2018|109195|
|2015| 94617|
|2019| 72449|
|2020|   166|
+----+------+

In [108]:
# top n publishers 

repartitioned_df.groupBy(col('Publisher'))\
.count()\
.orderBy(col('count').desc_nulls_last())\
.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Publisher='Little, Brown and Company,', count=16015), Row(Publisher='DK Publishing,', count=10844), Row(Publisher='Candlewick Press,', count=10744), Row(Publisher='Alfred A. Knopf,', count=9169), Row(Publisher='Thorndike Press,', count=7869)]

In [124]:
## write repartitioned_df back to S3 and parition by year column

## this will store in S3 in partitions e.g.
# library_inventory.parquet/Year=2015/part-00000-67b154e2-21c2-4019-b8a7-f1c6e6bd9f70.c000.snappy.parquet
# library_inventory.parquet/Year=2017/part-00000-67b154e2-21c2-4019-b8a7-f1c6e6bd9f70.c000.snappy.parquet

df_write = repartitioned_df.coalesce(1)
path = "s3://big-datasets-over-gb/seattle_library/processed/"
df_write.write.partitionBy("Year").format("parquet").save(f"{path}/library_inventory.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…